In [1]:
from api_keys import weather_api_key
import pandas as pd
from sqlalchemy import create_engine
import requests
import datetime

# Extraction

In [2]:
csv_file = "California_Fire_Incidents.csv"
cal_Fire_Inci_df = pd.read_csv(csv_file)
cal_Fire_Inci_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [3]:
csv_file = "County_info.csv"
county_info = pd.read_csv(csv_file)
county_info

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,1996-01-31,...,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31
0,3101,0,Los Angeles County,County,CA,CA,Los Angeles-Long Beach-Anaheim,6,37,175538.0,...,631450.0,633373.0,636620.0,639983.0,645447.0,650877.0,655589.0,657455.0,658836.0,664094.0
1,139,1,Cook County,County,IL,IL,Chicago-Naperville-Elgin,17,31,151180.0,...,248414.0,248122.0,247915.0,247925.0,248136.0,248548.0,248838.0,249194.0,249714.0,250814.0
2,1090,2,Harris County,County,TX,TX,Houston-The Woodlands-Sugar Land,48,201,100916.0,...,200198.0,200692.0,201230.0,201360.0,201790.0,202689.0,203762.0,204598.0,205096.0,205702.0
3,2402,3,Maricopa County,County,AZ,AZ,Phoenix-Mesa-Scottsdale,4,13,117969.0,...,292676.0,294735.0,297121.0,299473.0,301627.0,304151.0,307591.0,310595.0,313263.0,315783.0
4,2841,4,San Diego County,County,CA,CA,San Diego-Carlsbad,6,73,184765.0,...,612972.0,615771.0,619048.0,623541.0,627005.0,630889.0,633214.0,636054.0,637310.0,639238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,425,3101,Camas County,County,ID,ID,Hailey,16,25,NaN,...,120391.0,121292.0,122184.0,123119.0,123611.0,123799.0,124029.0,124508.0,125039.0,125683.0
2853,1648,3102,Daggett County,County,UT,UT,NaN,49,9,NaN,...,209722.0,210257.0,210202.0,210193.0,210127.0,210629.0,210902.0,211365.0,211801.0,212864.0
2854,2345,3112,Hinsdale County,County,CO,CO,NaN,8,53,NaN,...,330363.0,331253.0,331744.0,332152.0,332593.0,333098.0,332018.0,331755.0,331616.0,332733.0
2855,1933,3122,Mineral County,County,CO,CO,NaN,8,79,NaN,...,283733.0,283414.0,282863.0,282891.0,283029.0,283264.0,283064.0,283001.0,282851.0,282782.0


In [4]:
print(cal_Fire_Inci_df.columns)
print(county_info.columns)

Index(['AcresBurned', 'Active', 'AdminUnit', 'AirTankers', 'ArchiveYear',
       'CalFireIncident', 'CanonicalUrl', 'ConditionStatement',
       'ControlStatement', 'Counties', 'CountyIds', 'CrewsInvolved', 'Dozers',
       'Engines', 'Extinguished', 'Fatalities', 'Featured', 'Final',
       'FuelType', 'Helicopters', 'Injuries', 'Latitude', 'Location',
       'Longitude', 'MajorIncident', 'Name', 'PercentContained',
       'PersonnelInvolved', 'Public', 'SearchDescription', 'SearchKeywords',
       'Started', 'Status', 'StructuresDamaged', 'StructuresDestroyed',
       'StructuresEvacuated', 'StructuresThreatened', 'UniqueId', 'Updated',
       'WaterTenders'],
      dtype='object')
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS', '1996-01-31',
       ...
       '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
       '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30', '2020-0

In [5]:
# cal_Fire_Inci_df['Updated'] = pd.to_datetime(cal_Fire_Inci_df['Updated'])
# cal_Fire_Inci_df['Started'] = pd.to_datetime(cal_Fire_Inci_df['Started'])

# Transformation

In [6]:
CA_countys = county_info.loc[county_info['State'] == 'CA']
CA_countys['RegionName'] = CA_countys['RegionName'].str.extract(r'([\w ]*) County',expand=True)

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Drop unneeded data columns

In [7]:
first_date = "2012-12-31"
wrong_columns = CA_countys.loc[:,'Metro':first_date]
CA_countys = CA_countys.drop(columns = wrong_columns.columns)
end_date = "2020-01-31"
wrong_columns = CA_countys.loc[:,end_date:]
CA_countys = CA_countys.drop(columns = wrong_columns.columns)
CA_countys = CA_countys.drop(columns = ['SizeRank','RegionType','StateName'])

## Split normalize dataframe

In [8]:
countys = CA_countys['RegionName'].reset_index()
countys.to_csv('Countys.csv')

columns = CA_countys.loc[:,"2013-01-31":"2019-12-31"].columns
county_CA = pd.melt(CA_countys, id_vars=['RegionID', 'RegionName'], value_vars = columns)
county_CA

,RegionID,RegionName,variable,value
0,3101,Los Angeles,2013-01-31,390772.0
1,2841,San Diego,2013-01-31,395630.0
2,1286,Orange,2013-01-31,512190.0
3,2832,Riverside,2013-01-31,241946.0
4,3250,San Bernardino,2013-01-31,195566.0
...,...,...,...,...
4867,253,Mono,2019-12-31,460224.0
4868,2056,Trinity,2019-12-31,280572.0
4869,647,Modoc,2019-12-31,130233.0
4870,1391,Sierra,2019-12-31,239567.0


In [1]:
# cal_Fire_Inci_df[['UniqueId','CountyIds', 'ArchiveYear','AcresBurned',
#                                'StructuresDamaged', 'StructuresDestroyed', 'Updated',
#                                'Latitude', 'Longitude', 'Started']].head()

In [ ]:
Cal_Fire_db = cal_Fire_Inci_df[['UniqueId','CountyIds', 'ArchiveYear','AcresBurned',
                               'StructuresDamaged', 'StructuresDestroyed', 'Updated',
                               'Latitude', 'Longitude', 'Started']]
Cal_Fire_db = Cal_Fire_db.rename(columns = {'UniqueId':'FireID', 'ArchiveYear':'YearStarted', 'CountyIds':'CountyID',
       'Updated':'Extinguished'})
county_CA = county_CA.rename(columns = {'RegionID':'SaleID', 'variable':'Date', 'RegionName':'CountyID',
       'value':'MedianPrice'})
countys = countys.rename(columns ={'index': 'CountyID', 'RegionName': 'countyName'})

In [ ]:
Cal_Fire_db['FireID'] = Cal_Fire_db['FireID'].apply(lambda s : abs(hash(s)) % (10 ** 8))
Cal_Fire_db['CountyID'] = Cal_Fire_db['CountyID'].apply(lambda s : abs(hash(s)) % (10 ** 8))

In [ ]:
county_CA['SaleID'] = county_CA['MedianPrice'].apply(lambda s : abs(hash(s)) % (10 ** 8))
county_CA['CountyID'] = county_CA['CountyID'].apply(lambda s : abs(hash(s)) % (10 ** 8))

In [10]:
countys['CountyID'] = countys['countyName'].apply(lambda s : abs(hash(s)) % (10 ** 8))

In [11]:
Cal_Fire_db = Cal_Fire_db.fillna(0)

In [ ]:
rds_connection_string = "postgres:Tgnome75@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
countys.to_sql(name='county', con=engine, if_exists='replace', index=False)
county_CA.to_sql(name='salehistory', con=engine, if_exists='replace', index=False)
Cal_Fire_db.to_sql(name='fires', con=engine, if_exists='replace', index=False)

In [13]:
County = pd.read_sql_query('select * from county', con=engine)
SaleHistory = pd.read_sql_query('select * from salehistory', con=engine)
Fires = pd.read_sql_query('select * from fires', con=engine)

In [ ]:
County

In [ ]:
countyPrices = pd.merge(SaleHistory, County, on = 'CountyID', how='right', validate = 'many_to_one')

In [14]:
countyFires = pd.merge(Fires, County, on ='CountyID', how = 'right', validate = 'many_to_one')

,FireID,CountyID,YearStarted,AcresBurned,StructuresDamaged,StructuresDestroyed,Extinguished,Latitude,Longitude,Started
0,36110819,10606221,2013,257314.0,0.0,0.0,2013-09-06T18:30:00Z,37.857000,-120.086000,2013-08-17T15:25:00Z
1,13216448,93315353,2013,30274.0,0.0,0.0,2013-06-08T18:30:00Z,34.585595,-118.423176,2013-05-30T15:28:00Z
2,42492478,66065086,2013,27531.0,0.0,0.0,2013-07-30T18:00:00Z,33.709500,-116.728850,2013-07-15T13:43:00Z
3,30542050,89184188,2013,27440.0,0.0,0.0,2013-08-30T08:00:00Z,39.120000,-120.650000,2013-08-10T16:30:00Z
4,10527957,16754155,2013,24251.0,6.0,10.0,2013-05-11T06:30:00Z,0.000000,0.000000,2013-05-02T07:01:00Z
...,...,...,...,...,...,...,...,...,...,...
1631,46058626,66065086,2019,9.0,0.0,0.0,2019-10-10T18:11:59.733Z,33.827979,-117.499619,2019-10-10T12:08:00Z
1632,60201972,45413584,2019,2.0,0.0,0.0,2019-06-30T15:52:01.023Z,39.409722,-121.000556,2019-06-28T15:03:04Z
1633,75988724,83844217,2019,0.0,0.0,0.0,2019-12-03T16:35:20.93Z,38.734634,-121.729691,2019-11-25T12:02:02Z
1634,94089197,69682287,2019,0.0,0.0,0.0,2019-11-21T12:21:28.58Z,33.351145,-117.403719,2019-10-22T19:20:44Z


In [15]:
# Summary of crews
# Summary by year
# Summary by counties
# Summary by 
# year = 2013
# for index,row in cal_Fire_Inci_df[['Latitude','Longitude', 'ArchiveYear','Counties']].iterrows():
#     if row[2] != year:
#         year += 1
#     try:
#         url = f'https://history.openweathermap.org/data/2.5/aggregated/year={year}lat={row[0]}&lon={row[1]}&appid={weather_api_key}&units=metric'
        
#         city_data = requests.get(url).json()
# #         print(f'Processing Record {record_counter} of Set {set_counter} | {row[3]} County')
#         print(city_data.jsonify())
# #         city_data_list.append([{row},city_data['coord']['lat'],city_data['coord']['lon'],
# #                                city_data['main']['temp_max'],city_data['main']['humidity'],
# #                                city_data['clouds']['all'], city_data['wind']['speed'],
# #                                city_data['sys']['country'],city_data['dt']])
#     except:
#         print('City not found. Skipping...')
#         print(city_data)
#     break;



# year = 2013
# for index,row in cal_Fire_Inci_df[['Latitude','Longitude', 'ArchiveYear','Counties']].iterrows():
#     if row[2] != year:
#         year += 1
#     try:
#         url = f'https://history.openweathermap.org/data/2.5/aggregated/year=?lat={lat}&lon={lon}&appid={weather_api_key}&units=metric'
# #         http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={YOUR_API_KEY}

#         city_data = requests.get(url).json()
#         print(f'Processing Record {record_counter} of Set {set_counter} | {city}')
#         city_data_list.append([city,city_data['coord']['lat'],city_data['coord']['lon'],
#                                city_data['main']['temp_max'],city_data['main']['humidity'],
#                                city_data['clouds']['all'], city_data['wind']['speed'],
#                                city_data['sys']['country'],city_data['dt']])
#     except:
#         print('City not found. Skipping...')
#         print(city_data)